In [1]:
# -*- coding: utf-8 -*-
import os, sys
import glob
import json
sys.path.append("..")
sys.path.append("../..")
import dataset_utils.utils as utils
import dataset_utils.config as config
from utils.FaceAligner import FaceAligner

In [2]:
src_dir = os.path.join(config.DS_DIALOG, "transformed_split")
tgt_dir = os.path.join(config.DS_DIALOG, "face")
movie_files = ["01_01.MP4", "01_02.MP4", "02_01.MP4", "02_02.MP4", "03_01.MP4", "03_02.MP4"]
json_dir = os.path.join(config.DS_DIALOG, "face_bb_and_landmarks")

In [ ]:
for movie_file in movie_files:
    print(movie_file)
    
    movie_id = movie_file.split(".")[0]
    src_sub_dir = os.path.join(src_dir, movie_id)
    tgt_sub_dir = os.path.join(tgt_dir, movie_id)
    if not os.path.exists(tgt_sub_dir):
        os.makedirs(tgt_sub_dir)
    if not os.path.exists(json_dir):
        os.makedirs(json_dir)
    json_path = os.path.join(json_dir, "%s.json" % movie_id)
        
    aligner = FaceAligner()

    src_paths = sorted(glob.glob(os.path.join(src_sub_dir, "*.png")))
    results = {}
    for i, path in enumerate(src_paths):

        fname = os.path.basename(path)
        fid, ext = tuple(fname.split("."))
        
        # 途中から
        if movie_id == "01_01" and int(fid.split("_")[0]) < 3600:
            continue
        
        img = utils.imread(path)
        bbs = aligner.bbs(img=img)

        if bbs is not None or len(bbs) < 1:
            results[fname] = []
            for i, bb in enumerate(bbs):
                result = {}
                landmarks = aligner.landmarks(img=img, bb=bb)
                # save face img
                face_img = img[bb.top():bb.bottom(), bb.left():bb.right()]
                if i > 0:
                    face_img_name = ".".join(["_".join([fid, str(i)]), ext])
                else:
                    face_img_name = fname
                utils.imwrite(os.path.join(tgt_sub_dir, face_img_name), face_img)
                result['bb'] = [bb.left(), bb.top(), bb.right(), bb.bottom()]
                result["landmarks"] = landmarks
                results[fname].append(result)

    with open(json_path, "w") as fr:
        json.dump(results, fr, sort_keys=True)
    utils.notify("detect_face (%s) done!!" % movie_id)
utils.notify("@nogaken detect_face done!!")